In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

**Imports**
---



In [23]:
import numpy as np
import pandas as pd
from pathlib import Path
import sys
import os
import glob
import multiprocessing as mp
from datetime import date


pd.options.mode.chained_assignment = None  # default='warn'

In [24]:
p = Path('C:/Users/alexd/Dropbox/FFE')

edge_file = p / 'FinnShapeEdges.parquet'
wind_file = p / 'Copy of GD_wind.csv'
folder = Path('F:/output')

In [89]:
# load dataaaa
wind_data = pd.read_csv(wind_file) 
edgelist = pd.read_parquet(edge_file, engine='pyarrow')

In [90]:
rngFile = edgelist[['source', 'IgnProbBld']]
rngFile.drop_duplicates(['source'], inplace=True)

In [94]:
rngFile.IgnProbBld.describe()
rngFile.source.describe()

count    69824.000000
mean         0.000129
std          0.000223
min          0.000024
25%          0.000061
50%          0.000092
75%          0.000124
max          0.005239
Name: IgnProbBld, dtype: float64

count    69824.000000
mean     36867.715742
std      21298.611168
min          0.000000
25%      18424.750000
50%      36860.500000
75%      55315.250000
max      73734.000000
Name: source, dtype: float64

**Definitions & parallel computing**
---

In [28]:
# num_cores = multiprocessing.cpu_count()
# print(num_cores)
# # need to do this on Shell / Terminal:
# conda install ipyparallel
# ipcluster nbextension enable --user
# ipcluster start # or ipcluster start -n 4

In [29]:
import ipyparallel as ipp
client = ipp.Client()
dview = client[:]
client.ids

[0, 1, 2, 3, 4, 5]

In [30]:
# add variables to all engines
dview["edgelist"]=edgelist
dview["rngFile"]=rngFile
dview["wind_data"]=wind_data
dview["folder"]=folder

# add all libraries to engines
with dview.sync_imports():
    import numpy as np
    import pandas as pd
    from pathlib import Path
    import sys
    import os
    import glob
    from datetime import date

importing numpy on engine(s)
importing pandas on engine(s)
importing Path from pathlib on engine(s)
importing sys on engine(s)
importing os on engine(s)
importing glob on engine(s)
importing date from datetime on engine(s)


In [108]:
%%px

def wind_scenario(wind_data):
      import numpy as np
      i = np.random.randint(0, wind_data.values.shape[0])
      w = wind_data.values[i, 2]
      dist = wind_data.values[i, 1]
      b = wind_data.values[i, 3]
      bear_max = b + 45  # wind direction
      bear_min = b - 45
      if b == 360:
          bear_max = 45
      if b <= 0:  # should not be necessary
          bear_min = 0
      if b == 999:
          bear_max = 999
          bear_min = 0
      return bear_max, bear_min, dist # wind characteristics, bearing and distance


def ignition(rngList=rngFile, edges=edgelist):
    import numpy as np
    import pandas as pd
    rngList['rng'] = np.random.uniform(0, 1, size=rngList.values.shape[0])
    rngList = rngList[rngList['rng'] < rngList['IgnProbBld']]
    initialIgnitions = len(rngList)
    NewActiveEdges = edges[edges['source'].isin(rngList['source'])]
    return NewActiveEdges, initialIgnitions


def mask(t, activeEdges_d, listActivatedSources_d, w_b_max, w_b_min, w_d):
    import numpy as np
    if t==0: # special case at time=0
        return activeEdges_d
    else:
        mask = (activeEdges_d.bearing.values < w_b_max) & (activeEdges_d.bearing.values < w_b_min) & (activeEdges_d.distance < w_d)
        NewActiveEdges = activeEdges_d[mask]
        NewActiveEdges = NewActiveEdges[~NewActiveEdges.source.isin(listActivatedSources_d)]
        return NewActiveEdges


def propagation(activeEdges_d, edges=edgelist):
    import numpy as np
    import pandas as pd
    NewActiveEdges = edges[edges.source.isin(activeEdges_d.target)]
    return NewActiveEdges


In [127]:
@dview.parallel(block = False) # The @parallel decorator breaks up elementwise operations and distributes them.
def ffe_runs(n):
    import numpy as np
    import pandas as pd
    from datetime import date
    for scenario in n:
        # initial setup
        listActivatedSources = []
        listScenarioDataframes = []
        condition = True
        time = 0 
        # wind conditions
        w_bearing_max, w_bearing_min, w_distance = wind_scenario(wind_data)
        # ignition / initial state and edges selection
        ActiveEdges, numberIgnitions = ignition()
        if ActiveEdges.empty:
            print(f"no ignitions {numberIgnitions}")
            condition = False
            continue
        while condition: # spread burn zone
            ActiveEdges = mask(time, ActiveEdges, listActivatedSources, w_bearing_max, w_bearing_min, w_distance)
            if ActiveEdges.empty: #no more buildings to burn
                break
            burns = ActiveEdges.drop_duplicates(['source'], inplace=False)
#             print(f"Active edges {len(ActiveEdges)} / no duplicate = {len(burns)}")
            burns['time'] = time
            listScenarioDataframes.append(burns)
            listActivatedSources.extend(ActiveEdges.source.values)
            ActiveEdges = propagation(ActiveEdges)
            time += 1
        
        print(f'finishing scenario --- {scenario} time ---- {time} \n started with {numberIgnitions} ignitions ')

        Activations = pd.concat(listScenarioDataframes)
        Activations["scenario"] = scenario
        Activations["InitialIgnitions"] = numberIgnitions
        Activations.to_parquet(str(folder) + '/' + f'scenario{scenario}_{str(date.today())}.parquet', engine='auto', compression="GZIP")
        

**Main**
---


---



In [128]:
%%time
ffe_runs(range(3000))

finishing scenario --- 0 time ---- 38 
 started with 7 ignitions 
Wall time: 7.94 s



**Backup**
---



---



In [130]:
# test = pd.read_parquet(folder / 'scenario0_2020-08-06.parquet')
# test

,source,target,distance,bearing,IgnProbBld,time,scenario,InitialIgnitions
256459,5895,1579,72.341491,64.134984,0.000236,0,0,7
359312,8299,1049,28.437373,80.310205,0.000438,0,0,7
1081448,25109,1095,60.216590,157.301388,0.000106,0,0,7
1454959,33783,6458,76.268136,213.489524,0.005239,0,0,7
1998807,46317,444,62.253573,0.818445,0.000127,0,0,7
...,...,...,...,...,...,...,...,...
3018908,70098,68652,4.960175,235.413575,0.000322,35,0,7
403429,9326,10806,15.074119,205.333995,0.000322,36,0,7
1008950,23405,58170,5.750335,212.989010,0.000322,36,0,7
466790,10806,9326,15.074119,25.333995,0.000322,37,0,7


In [131]:
# def clean_up(path=folder):
#     files = path.glob('*scenario*')
#     for f in files:
#         print(f)
#         os.remove(f)

In [132]:
# clean_up()

F:\output\scenario0_2020-08-06.parquet
F:\output\scenario1000_2020-08-06.parquet
F:\output\scenario1001_2020-08-06.parquet
F:\output\scenario1002_2020-08-06.parquet
F:\output\scenario1003_2020-08-06.parquet
F:\output\scenario1004_2020-08-06.parquet
F:\output\scenario1005_2020-08-06.parquet
F:\output\scenario1006_2020-08-06.parquet
F:\output\scenario1007_2020-08-06.parquet
F:\output\scenario1008_2020-08-06.parquet
F:\output\scenario1009_2020-08-06.parquet
F:\output\scenario100_2020-08-06.parquet
F:\output\scenario1010_2020-08-06.parquet
F:\output\scenario1011_2020-08-06.parquet
F:\output\scenario1012_2020-08-06.parquet
F:\output\scenario1013_2020-08-06.parquet
F:\output\scenario1014_2020-08-06.parquet
F:\output\scenario1015_2020-08-06.parquet
F:\output\scenario1016_2020-08-06.parquet
F:\output\scenario1017_2020-08-06.parquet
F:\output\scenario1018_2020-08-06.parquet
F:\output\scenario1019_2020-08-06.parquet
F:\output\scenario101_2020-08-06.parquet
F:\output\scenario1020_2020-08-06.parqu

F:\output\scenario1298_2020-08-06.parquet
F:\output\scenario1299_2020-08-06.parquet
F:\output\scenario129_2020-08-06.parquet
F:\output\scenario12_2020-08-06.parquet
F:\output\scenario1300_2020-08-06.parquet
F:\output\scenario1301_2020-08-06.parquet
F:\output\scenario1302_2020-08-06.parquet
F:\output\scenario1303_2020-08-06.parquet
F:\output\scenario1304_2020-08-06.parquet
F:\output\scenario1305_2020-08-06.parquet
F:\output\scenario1306_2020-08-06.parquet
F:\output\scenario1307_2020-08-06.parquet
F:\output\scenario1308_2020-08-06.parquet
F:\output\scenario1309_2020-08-06.parquet
F:\output\scenario130_2020-08-06.parquet
F:\output\scenario1310_2020-08-06.parquet
F:\output\scenario1311_2020-08-06.parquet
F:\output\scenario1312_2020-08-06.parquet
F:\output\scenario1313_2020-08-06.parquet
F:\output\scenario1314_2020-08-06.parquet
F:\output\scenario1315_2020-08-06.parquet
F:\output\scenario1316_2020-08-06.parquet
F:\output\scenario1317_2020-08-06.parquet
F:\output\scenario1318_2020-08-06.parq

F:\output\scenario1593_2020-08-06.parquet
F:\output\scenario1594_2020-08-06.parquet
F:\output\scenario1595_2020-08-06.parquet
F:\output\scenario1596_2020-08-06.parquet
F:\output\scenario1597_2020-08-06.parquet
F:\output\scenario1598_2020-08-06.parquet
F:\output\scenario1599_2020-08-06.parquet
F:\output\scenario159_2020-08-06.parquet
F:\output\scenario15_2020-08-06.parquet
F:\output\scenario1600_2020-08-06.parquet
F:\output\scenario1601_2020-08-06.parquet
F:\output\scenario1602_2020-08-06.parquet
F:\output\scenario1603_2020-08-06.parquet
F:\output\scenario1604_2020-08-06.parquet
F:\output\scenario1605_2020-08-06.parquet
F:\output\scenario1606_2020-08-06.parquet
F:\output\scenario1607_2020-08-06.parquet
F:\output\scenario1608_2020-08-06.parquet
F:\output\scenario1609_2020-08-06.parquet
F:\output\scenario160_2020-08-06.parquet
F:\output\scenario1610_2020-08-06.parquet
F:\output\scenario1611_2020-08-06.parquet
F:\output\scenario1612_2020-08-06.parquet
F:\output\scenario1613_2020-08-06.parq

F:\output\scenario1957_2020-08-06.parquet
F:\output\scenario1958_2020-08-06.parquet
F:\output\scenario1959_2020-08-06.parquet
F:\output\scenario195_2020-08-06.parquet
F:\output\scenario1960_2020-08-06.parquet
F:\output\scenario1961_2020-08-06.parquet
F:\output\scenario1962_2020-08-06.parquet
F:\output\scenario1963_2020-08-06.parquet
F:\output\scenario1964_2020-08-06.parquet
F:\output\scenario1965_2020-08-06.parquet
F:\output\scenario1966_2020-08-06.parquet
F:\output\scenario1967_2020-08-06.parquet
F:\output\scenario1968_2020-08-06.parquet
F:\output\scenario1969_2020-08-06.parquet
F:\output\scenario196_2020-08-06.parquet
F:\output\scenario1970_2020-08-06.parquet
F:\output\scenario1971_2020-08-06.parquet
F:\output\scenario1972_2020-08-06.parquet
F:\output\scenario1973_2020-08-06.parquet
F:\output\scenario1974_2020-08-06.parquet
F:\output\scenario1975_2020-08-06.parquet
F:\output\scenario1976_2020-08-06.parquet
F:\output\scenario1977_2020-08-06.parquet
F:\output\scenario1978_2020-08-06.pa

F:\output\scenario229_2020-08-06.parquet
F:\output\scenario22_2020-08-06.parquet
F:\output\scenario2300_2020-08-06.parquet
F:\output\scenario2301_2020-08-06.parquet
F:\output\scenario2302_2020-08-06.parquet
F:\output\scenario2303_2020-08-06.parquet
F:\output\scenario2304_2020-08-06.parquet
F:\output\scenario2305_2020-08-06.parquet
F:\output\scenario2306_2020-08-06.parquet
F:\output\scenario2307_2020-08-06.parquet
F:\output\scenario2308_2020-08-06.parquet
F:\output\scenario2309_2020-08-06.parquet
F:\output\scenario230_2020-08-06.parquet
F:\output\scenario2310_2020-08-06.parquet
F:\output\scenario2311_2020-08-06.parquet
F:\output\scenario2312_2020-08-06.parquet
F:\output\scenario2313_2020-08-06.parquet
F:\output\scenario2314_2020-08-06.parquet
F:\output\scenario2315_2020-08-06.parquet
F:\output\scenario2316_2020-08-06.parquet
F:\output\scenario2317_2020-08-06.parquet
F:\output\scenario2318_2020-08-06.parquet
F:\output\scenario2319_2020-08-06.parquet
F:\output\scenario231_2020-08-06.parqu

F:\output\scenario2679_2020-08-06.parquet
F:\output\scenario267_2020-08-06.parquet
F:\output\scenario2680_2020-08-06.parquet
F:\output\scenario2681_2020-08-06.parquet
F:\output\scenario2682_2020-08-06.parquet
F:\output\scenario2683_2020-08-06.parquet
F:\output\scenario2684_2020-08-06.parquet
F:\output\scenario2685_2020-08-06.parquet
F:\output\scenario2686_2020-08-06.parquet
F:\output\scenario2687_2020-08-06.parquet
F:\output\scenario2688_2020-08-06.parquet
F:\output\scenario2689_2020-08-06.parquet
F:\output\scenario268_2020-08-06.parquet
F:\output\scenario2690_2020-08-06.parquet
F:\output\scenario2691_2020-08-06.parquet
F:\output\scenario2692_2020-08-06.parquet
F:\output\scenario2693_2020-08-06.parquet
F:\output\scenario2694_2020-08-06.parquet
F:\output\scenario2695_2020-08-06.parquet
F:\output\scenario2696_2020-08-06.parquet
F:\output\scenario2697_2020-08-06.parquet
F:\output\scenario2698_2020-08-06.parquet
F:\output\scenario2699_2020-08-06.parquet
F:\output\scenario269_2020-08-06.par

F:\output\scenario2991_2020-08-06.parquet
F:\output\scenario2992_2020-08-06.parquet
F:\output\scenario2993_2020-08-06.parquet
F:\output\scenario2994_2020-08-06.parquet
F:\output\scenario2995_2020-08-06.parquet
F:\output\scenario2996_2020-08-06.parquet
F:\output\scenario2997_2020-08-06.parquet
F:\output\scenario2998_2020-08-06.parquet
F:\output\scenario2999_2020-08-06.parquet
F:\output\scenario299_2020-08-06.parquet
F:\output\scenario29_2020-08-06.parquet
F:\output\scenario2_2020-08-06.parquet
F:\output\scenario300_2020-08-06.parquet
F:\output\scenario301_2020-08-06.parquet
F:\output\scenario302_2020-08-06.parquet
F:\output\scenario303_2020-08-06.parquet
F:\output\scenario304_2020-08-06.parquet
F:\output\scenario305_2020-08-06.parquet
F:\output\scenario306_2020-08-06.parquet
F:\output\scenario307_2020-08-06.parquet
F:\output\scenario308_2020-08-06.parquet
F:\output\scenario309_2020-08-06.parquet
F:\output\scenario30_2020-08-06.parquet
F:\output\scenario310_2020-08-06.parquet
F:\output\s

F:\output\scenario50_2020-08-06.parquet
F:\output\scenario510_2020-08-06.parquet
F:\output\scenario511_2020-08-06.parquet
F:\output\scenario512_2020-08-06.parquet
F:\output\scenario513_2020-08-06.parquet
F:\output\scenario514_2020-08-06.parquet
F:\output\scenario515_2020-08-06.parquet
F:\output\scenario516_2020-08-06.parquet
F:\output\scenario517_2020-08-06.parquet
F:\output\scenario518_2020-08-06.parquet
F:\output\scenario519_2020-08-06.parquet
F:\output\scenario51_2020-08-06.parquet
F:\output\scenario520_2020-08-06.parquet
F:\output\scenario521_2020-08-06.parquet
F:\output\scenario522_2020-08-06.parquet
F:\output\scenario523_2020-08-06.parquet
F:\output\scenario524_2020-08-06.parquet
F:\output\scenario525_2020-08-06.parquet
F:\output\scenario526_2020-08-06.parquet
F:\output\scenario527_2020-08-06.parquet
F:\output\scenario528_2020-08-06.parquet
F:\output\scenario529_2020-08-06.parquet
F:\output\scenario52_2020-08-06.parquet
F:\output\scenario530_2020-08-06.parquet
F:\output\scenario5

F:\output\scenario871_2020-08-06.parquet
F:\output\scenario872_2020-08-06.parquet
F:\output\scenario873_2020-08-06.parquet
F:\output\scenario874_2020-08-06.parquet
F:\output\scenario875_2020-08-06.parquet
F:\output\scenario876_2020-08-06.parquet
F:\output\scenario877_2020-08-06.parquet
F:\output\scenario878_2020-08-06.parquet
F:\output\scenario879_2020-08-06.parquet
F:\output\scenario87_2020-08-06.parquet
F:\output\scenario880_2020-08-06.parquet
F:\output\scenario881_2020-08-06.parquet
F:\output\scenario882_2020-08-06.parquet
F:\output\scenario883_2020-08-06.parquet
F:\output\scenario884_2020-08-06.parquet
F:\output\scenario885_2020-08-06.parquet
F:\output\scenario886_2020-08-06.parquet
F:\output\scenario887_2020-08-06.parquet
F:\output\scenario888_2020-08-06.parquet
F:\output\scenario889_2020-08-06.parquet
F:\output\scenario88_2020-08-06.parquet
F:\output\scenario890_2020-08-06.parquet
F:\output\scenario891_2020-08-06.parquet
F:\output\scenario892_2020-08-06.parquet
F:\output\scenario